__Final Project__

# __Data Collection Process:__

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
import random
import os
from sklearn.preprocessing import MinMaxScaler
from spotipy import Spotify 
from typing import List
import oauth2
from dotenv import load_dotenv

### __Loading Client Information:__

In [5]:
client_id = '5477354bc87a4af1b2b8b10e23998f8d'
client_secret = '124885a709a44f5b9f4c251458e19a22'

sp_cred = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)  
sp = spotipy.Spotify(auth_manager=sp_cred)  

# sp_oauth = SpotifyOAuth(client_id=client_id, client_secret=client_secret, )
# token_info = sp_oauth.get_cached_token() 
# if not token_info:
#     auth_url = sp_oauth.get_authorize_url(show_dialog=True)
#     print(auth_url)
#     response = input('Paste the above link into your browser, then paste the redirect url here: ')

#     code = sp_oauth.parse_response_code(response)
#     token_info = sp_oauth.get_access_token(code)

#     token = token_info['access_token']

# sp = spotipy.Spotify(auth=token)

### __Collecting Artist Data:__

In [6]:
def get_random_artists(limit=20, min_popularity=20, 
                                max_popularity=100):
    artist_list = []

    search_queries = [
        'cu', 'vu', 'bu', 'wu', 'ru', 
        'pu', 'qu', 'su', 'lu', 'nu']

    for query in search_queries:
        results = sp.search(q=f'artist:{query}', 
                            type='artist', market='US', 
                            limit=50, 
                            offset=random.randint(0, 950))
        for artist in results['artists']['items']:
            if min_popularity <= artist['popularity'] \
                <= max_popularity:
                artist_list.append(artist['name'])
                if len(artist_list) >= limit:
                    return artist_list[:limit]

    return artist_list[:limit]

# Fetch random artists
artists = get_random_artists()
print(artists)


['Frente Cumbiero', 'Mr Cutts', 'Curli Loxx', 'Orquestra Românticos de Cuba', 'Curly Russell', 'Culturas', 'CubeNatural', 'Cuarenta y Cinco', 'Boy Called Cute', 'Curlybamm', 'Stan Curley', 'Felix Cumbe', 'Los Violines Románticos De Cuba', 'Curtis Lee', 'Cummrs', 'The Destruction Of The Cult Of The Sun', 'Cuper', 'DJ Cummerbund', 'Ismael Fernandez De La Cuesta', 'Canciones De Cuna Para Dormir Bebes']


### __Collecting Song Data:__

In [8]:
songs_data = []
for artist in artists:
    results = sp.search(q=f'artist:{artist}', 
                        type='track', limit=10)
    for track in results['tracks']['items']:

        # Get audio features for the track
        audio_features = sp.audio_features(track['id'])[0]

        # Get the popularity of the song and the artist
        song_name = track['name']
        song_popularity = track['popularity']
        artist_popularity = results['tracks']['items'][0]['popularity']
        
        print(f"track: {song_name}")

        # Exclude the main artist
        featured_artists = track['artists'][1:]  

        # Get the popularity of the first featured artist
        if featured_artists:
            first_featured_artist = featured_artists[0]['name']
            
            search_result = sp.search(q=f'artist:\
                    {first_featured_artist}', type='artist')
            if search_result['artists']['items']:
                items = search_result['artists']['items']
                
                first_featured_artist_popularity = items[0]\
                    ['popularity']
            else:
                first_featured_artist_popularity = None
        else:
            first_featured_artist = None
            first_featured_artist_popularity = None

        # Add the audio features
        if audio_features:
            id = track['id'],
            album = track['album']['name'],
            release_date = track['album']['release_date'],
            duration = track['duration_ms'],
            explicit = track['explicit'],
            danceability = audio_features['danceability'],
            energy = audio_features['energy'],
            key = audio_features['key'],
            loudness = audio_features['loudness'],
            mode = audio_features['mode'],
            speechiness = audio_features['speechiness'],
            acousticness = audio_features['acousticness'],
            instrumentalness = audio_features['instrumentalness'],
            liveness = audio_features['liveness'],
            valence = audio_features['valence'],
            tempo = audio_features['tempo']
        else:
            id = None
            album = None
            release_date = None
            duration = None
            explicit = None
            danceability = None
            energy = None
            key = None
            loudness = None
            mode = None
            speechiness = None
            acousticness = None
            instrumentalness = None
            liveness = None
            valence = None
            tempo = None
        
        # Add the data to the list
        songs_data.append({
            'Artist': artist,
            'Song': song_name,
            'Song Popularity': song_popularity,
            'Artist Popularity': artist_popularity,
            'Featured Artist': first_featured_artist,
            'Featured Artist Popularity': 
            first_featured_artist_popularity,
            'Track ID': id,
            'Album Name': album,
            'Release Date': release_date,
            'Duration (ms)': duration,
            'Explicit': explicit,
            'Danceability': danceability,
            'Energy': energy,
            'Key': key,
            'Loudness': loudness,
            'Mode': mode,
            'Speechiness': speechiness,
            'Acousticness': acousticness,
            'Instrumentalness': instrumentalness,
            'Liveness': liveness,
            'Valence': valence,
            'Tempo': tempo
            
        })

songs_df = pd.DataFrame(songs_data)

print(songs_df.head())

Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=1O9k2KJkdTALaxjkzUoLCT:
 Max Retries, reason: too many 429 error responses

In [77]:
songs_df.size

17424

### __Data Cleaning__

In [78]:
# Columns to exclude from cleaning
exclude_columns = ['Artist', 'Song', 'Song Popularity', 'Artist Popularity', 'Featured Artist', 'Featured Artist Popularity']

# Function to clean the data
def clean_data(value):
    if isinstance(value, str):
        value = value.replace("(", "").replace(")", "").replace("'", "").replace(",", "")
    return value

# Apply cleaning function to all columns except the excluded ones
for col in songs_df.columns:
    if col not in exclude_columns:
        # Convert column to string before applying the cleaning function
        songs_df[col] = songs_df[col].astype(str).apply(clean_data)

# Save the cleaned DataFrame to CSV
songs_df.to_csv('tracks1.csv', index=False)

print(songs_df)

                                     Artist                            Song  \
0                              Curtis Walsh                   Full Recovery   
1                              Curtis Walsh            Perfect Rain - Remix   
2                              Curtis Walsh                    Perfect Rain   
3                              Curtis Walsh       Seven Seas (Konmak Remix)   
4                              Curtis Walsh                      Seven Seas   
..                                      ...                             ...   
787  Los Internacionales Nenes de la Cumbia  Tu Abandono / Perdonarte Jamás   
788  Los Internacionales Nenes de la Cumbia                    Sufro por Ti   
789  Los Internacionales Nenes de la Cumbia                 Marcado - Remix   
790  Los Internacionales Nenes de la Cumbia                   Parrandita 88   
791  Los Internacionales Nenes de la Cumbia       Guitarra Chichera - Remix   

     Song Popularity  Artist Popularity Featured Ar

## __Writing to CSV__

In [ ]:
existing_df = pd.read_csv('tracks1.csv')
combined_df = pd.concat([existing_df, songs_df], ignore_index=True)
combined_df.to_csv('tracks.csv', index=False)

# Research Question

//

## Model 1

## Visualization for Model 1

## Model 2

## Visualization for Model 2

# EXPLAIN EACH SECTION!!!